In [27]:
import pandas as pd
#from urllib.request import urlretrieve
#import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
#import nbformat
#sns.set_style('darkgrid')
#matplotlib.rcParams['font.size'] = 14
#matplotlib.rcParams['figure.figsize'] = (10, 6)
#matplotlib.rcParams['figure.facecolor'] = '#00000000'
import numpy as np
#from sklearn.preprocessing import StandardScaler
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
#%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import time

In [28]:
torch.cuda.is_available()
def get_device():
    # Step 2: Check if a GPU is available
    if torch.cuda.is_available():
        # Step 3: Set the device to GPU
        device = torch.device("cuda")
        #device = torch.device("cpu")
        print("GPU is available. Using GPU.")
    else:
        # Step 3: Default to CPU
        device = torch.device("cpu")
        print("GPU is not available. Using CPU.")
    
    # Step 4: Return the device
    return device

# Example usage
device = get_device()

GPU is available. Using GPU.


In [29]:
class ThreeLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ThreeLayerNN, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.relu1 = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second fully connected layer
        self.relu2 = nn.ReLU()  # Activation function
        self.fc3 = nn.Linear(hidden_size, output_size)  # Third fully connected layer

    def forward(self, x):
        # Forward pass
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [30]:

num_samples = 1000
num_instances = 10000


x = torch.rand(num_instances, num_samples)
y = torch.rand(num_instances, num_samples)
z = torch.rand(num_instances, num_samples)
target = torch.randint(-1000, 1001, (num_instances, num_samples))
data = torch.stack((x, y, z, target), dim=2).to(device)




In [31]:
val_size = num_instances//10
train_size = num_instances - val_size
print(val_size)
print(train_size)
start_time = time.time()
train_ds, val_ds = random_split(data, [train_size, val_size])
batch_size=20
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=0)

1000
9000


In [32]:
#for batch in train_loader:
#    print(batch.shape)
#    break

In [33]:
#model = nn.Linear(100 * 3, 100)

In [34]:
#model.parameters()

In [35]:
#data[:,...,:-1].shape

In [36]:
model = ThreeLayerNN(3, 100, 1).to(device)

In [37]:
#model.parameters()

In [38]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming model, train_loader, and criterion are already defined
#model = ...  # Your neural network model
#train_loader = ...  # DataLoader for your training data
criterion = nn.MSELoss()  # Loss function for regression
optimizer = optim.Adam(model.parameters(), lr=1e-5)  # Optimizer

num_epochs = 10  # Number of epochs

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for batch in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        inputs = batch[:,...,:-1].to(device)
        outputs = model(inputs).to(device)
        targets =   batch[:,...,-1].to(device)   # Forward pass
        outputs = outputs.squeeze(-1)
        loss = criterion(outputs, targets)  # Compute the loss

        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for {num_instances} simulations with {num_samples} points each on {device}.")


Epoch [1/10], Loss: 333865.10020833334
Epoch [2/10], Loss: 333865.08416666667
Epoch [3/10], Loss: 333865.0677083333
Epoch [4/10], Loss: 333865.0554861111
Epoch [5/10], Loss: 333865.04243055556
Epoch [6/10], Loss: 333865.0320138889
Epoch [7/10], Loss: 333865.0238888889
Epoch [8/10], Loss: 333865.0159722222
Epoch [9/10], Loss: 333865.00930555555
Epoch [10/10], Loss: 333865.00083333335
Elapsed time: 42.95975923538208 seconds for 10000 simulations with 1000 points each on cuda.


In [40]:
model.eval()  # Set the model to evaluation mode
val_loss = 0.0
start_time = time.time()
with torch.no_grad():
    for batch in val_loader:
        inputs = batch[:, ..., :-1].to(device)
        outputs = model(inputs).to(device)
        targets = batch[:, ..., -1].to(device)  # Forward pass
        outputs = outputs.squeeze(-1)
        loss = criterion(outputs, targets).to(device)  # Compute the loss
        val_loss += loss.item()
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss/len(val_loader)}")
print(f"validation took {elapsed_time} secondson {device}.")

Epoch [10/10], Validation Loss: 333683.5175
validation took 0.15374302864074707 secondson cuda.
